In [2]:
import pandas as pd
import numpy as np

In [15]:
NAME = 'EXP_star'

df = pd.read_csv("./12movies_init_FaceReader_output/EXPERIMENT_filtered_star.csv", delimiter=',')
start = 1000
df[start:start+10]

,Unnamed: 0,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,...,AU07_r_filtered,AU09_r_filtered,AU12_r_filtered,AU14_r_filtered,AU15_r_filtered,AU17_r_filtered,AU20_r_filtered,AU23_r_filtered,AU25_r_filtered,AU26_r_filtered
1000,1000,5404,0,225.350,0.88,1,0.211196,0.214736,-0.953564,0.044719,...,0.000000,0.888975,0.510731,0.0,1.351582,0.749657,0.000000,0.000397,0.000000,1.944096
1001,1001,5405,0,225.392,0.88,1,0.206328,0.206563,-0.956431,0.039951,...,0.000000,0.874433,0.513833,0.0,1.326301,0.846767,0.000000,0.000000,0.000000,2.072468
1002,1002,5406,0,225.434,0.88,1,0.216782,0.193894,-0.956771,0.047286,...,0.000000,0.864696,0.511542,0.0,1.308154,0.954977,0.000000,0.000000,0.000000,2.212528
1003,1003,5407,0,225.475,0.88,1,0.212517,0.191424,-0.958224,0.043115,...,0.000000,0.863570,0.505225,0.0,1.294455,1.059415,0.000000,0.000000,0.000000,2.362470
1004,1004,5408,0,225.517,0.88,1,0.200748,0.191599,-0.960724,0.031296,...,0.000000,0.869746,0.498934,0.0,1.284197,1.151221,0.000000,0.000000,0.000000,2.507916
1005,1005,5409,0,225.559,0.88,1,0.196349,0.184739,-0.962974,0.021863,...,0.000000,0.876738,0.498318,0.0,1.279502,1.225840,0.000000,0.000286,0.000000,2.649967
1006,1006,5410,0,225.601,0.98,1,0.186641,0.177609,-0.966240,0.012721,...,0.000000,0.876243,0.505780,0.0,1.281080,1.273559,0.000000,0.000181,0.000000,2.781637
1007,1007,5411,0,225.642,0.88,1,0.178679,0.144412,-0.973252,-0.002096,...,0.000000,0.861666,0.521709,0.0,1.288739,1.283880,0.000000,0.000000,0.000000,2.901657
1008,1008,5412,0,225.684,0.88,1,0.180509,0.140572,-0.973476,0.002836,...,0.002977,0.829025,0.547171,0.0,1.299693,1.260513,0.001126,0.000684,0.000422,2.996246
1009,1009,5413,0,225.726,0.88,1,0.182479,0.170085,-0.968387,0.012552,...,0.003895,0.784061,0.575757,0.0,1.322183,1.219802,0.001663,0.000734,0.000579,3.081616


In [4]:
len(df)

121096

In [5]:
cols = df.columns

In [6]:
col_of_int = ['x_0', 'x_16', 'x_8', 'x_4', 'x_12', 'x_33', 'x_27', 'x_39', 'x_42', 'y_0', 'y_16', 'y_8', 'y_4', 'y_12', 'y_33', 'y_27', 'y_39', 'y_42']
#face_y = pd.DataFrame(df[0:500], columns=['y_0', 'y_8', 'y_16', 'y_4', 'y_12', 'y_33', 'y_27', 'y_39', 'y_42'])
#'frame', 'face_id', 

In [7]:
#get a vector of coordinates of two faces - dset1 and dset2 
#returns the sum of distances between related dots (e.g., x_8 from dset1 and x_8 from dset2)
def distance(dset1, dset2):

    distance = 0
    
    for i in range(len(dset1)):
        distance += abs(dset1[i] - dset2[i])
    
    return distance


In [8]:
#get a vector of coordinates of two faces - dset1 and dset2
#if False, the face in a folowing frame is too far from the face in a current dataset
def coord_match(dset1, dset2):
    match = True
    width = abs(dset1[0] - dset1[2])//2
    height = abs(dset1[1] - dset1[0])//2
    for i in range(len(dset1)//2):
        if (abs(dset1[i] - dset2[i]) > width) or (abs(dset1[i + len(dset1)//2] - dset2[i + len(dset1)//2]) > height):
            match = False
    return match


In [10]:
def face_fitness(dset):
    time_start = float(dset.iloc[0]['timestamp']) #first row
    time_end = float(dset.iloc[-1]['timestamp']) #last row

    if time_end - time_start < 3.0:
        print(time_end - time_start)
        return False
    
    success = list(dset['success'])

    '''for i in success:
        if int(i) == 0:
            return False
    '''
    return True

    


In [11]:
def compare(frame1, frame2):
    col_of_int = ['x_0', 'x_16', 'x_8', 'x_4', 'x_12', 'x_33', 'x_27', 'x_39', 
    'x_42', 'y_0', 'y_16', 'y_8', 'y_4', 'y_12', 'y_33', 'y_27', 'y_39', 'y_42']
    #calculate distances
    matches = {}
    to_delete = [] #id of datasets presented in frame1, but don't exist anymore in frame2
    to_add = [] #id of datasets presented in frame2, but absence in frame1

    arbit_key = list(frame2.keys())[0]
    
    for key1 in frame1: #walk trough dset1
        matched = -1
        #add = -1
        last_line1 = frame1[key1].iloc[-1] #take the last row from a dataset
        last_line2 = frame2[arbit_key].iloc[-1] #take the last row from a dataset
        min_dist = distance(last_line1[col_of_int], last_line2[col_of_int])
        #print(min_dist)

        for key2 in frame2:
            last_line1 = frame1[key1].iloc[-1]
            last_line2 = frame2[key2].iloc[-1]
            temp = distance(last_line1[col_of_int], last_line2[col_of_int])
            
            if temp <= min_dist and coord_match(last_line1[col_of_int], last_line2[col_of_int]): #dataset in presented in frame2
                min_dist = temp
                matched = key2 #then add this J dataset to I dataset from frame1
        if matched != -1:
            matches[key1] = matched
        
    #print(matches)
    keys1 = list(frame1.keys())
    keys2 = list(frame2.keys())

    matched_f1 = list(matches.keys())
    matched_f2 = list(matches.values())

    for i in keys1:
        if i not in matched_f1:
            to_delete.append(i)
    
    for i in keys2:
        if i not in matched_f2:
            to_add.append(i)
        
    return to_delete, to_add, matches


In [12]:
count = 0 #iterator
last_faceid = 0 #face in the final dataset
faceid = 0 #last face in the d0 (tracker)
faces = {} #dict of datasets

#cycle for the first frame data extraction. It will then be compared with the next frames
d0 = {}
minidata = df.iloc[count]
frame = minidata['frame']

#initial dataset consists of faces presented in the furst frame
while True:
    miniframe = df[count:count+1]
    minidata = df.iloc[count] #stupid way of getting the 'frame' column value
    if minidata['frame'] != frame:
        break
    d0[faceid] = miniframe
    faceid += 1 #last faceID in the 
    count += 1

#cycle for data analysis
#take another frame
while count < len(df)-1:  
    d1 = {}
    minidata = df.iloc[count]
    frame = int(minidata['frame'])

    while True:
        miniframe = df[count:count+1]
        minidata = df.iloc[count] #stupid way of getting the 'frame' column value
        id = int(minidata['face_id'])
        if minidata['frame'] != frame:
            break
        d1[id] = miniframe
        count += 1


    delete, add, matches = compare(d0, d1) #compare faces from the previous and the current frames
    print(matches, delete, add)

    #add data to d0 if a new face appears
    for i in add:
        d0[faceid] = d1[i]
        faceid += 1

    #face do not exist anymore in the new dataset
    #remove it from d0; do checkig - if fits, add it to the final vocab (faces)
    for i in delete:
        if face_fitness(d0[i]) == True:
            faces[last_faceid] = d0[i]
            last_faceid += 1
        #else:
            #faceid -= 1
        d0.pop(i, None)
    
    for i in matches:
        d0[i] = pd.concat([d0[i], d1[matches[i]]])

    
for i in d0:
        if face_fitness(d0[i]) == True:
            faces[last_faceid] = d0[i]
            last_faceid += 1

print(d0.keys())
    
    

{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] []
{0: 0} [] [1]
{0: 0, 1: 1} [] []
{1: 1} [0] []
1.3770000000000095
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] []
{1: 1} [] [

In [16]:
len(faces[0])

110

In [17]:
result_faces = {}
timestamps = []
face_id = 0
newkey = 0
total_time = 0

id = 0
for key in faces:
    print(key)
    data = faces[key]

    temp = data.iloc[0]
    time_start = temp['timestamp']
    frame_start = temp['frame']
    frame1 = 0
    time_end = temp['timestamp']
    frame_end = temp['frame']
    frame2 = 0

    count = 0

    new_dset = pd.DataFrame(columns=cols)
    flag = False

    while count < len(data)-1:
        temp = data.iloc[count]
        success = int(temp['success'])
        if success == 0:
            count += 1
            #temp = data.iloc[count]
            #success = int(temp['success'])
            #print(success, count)
            if count >= len(data)-1:
                break
            continue

        time_start = float(temp['timestamp'])
        frame_start = temp['frame']
        frame1 = count
        while success == 1:
            #print(success, count)
            temp = data.iloc[count]
            time_end = float(temp['timestamp'])
            frame_end = temp['frame']
            frame2 = count
            count += 1
            if count >= len(data)-1:
                break
            success = int(temp['success'])
        
        if time_end - time_start > 3.0:
            total_time += time_end - time_start
            new_dset = pd.concat([new_dset, faces[key][frame1:frame2]])
            flag = True
            print(time_start, time_end, frame1, frame2, frame_start, frame_end-1) #fram_end - is the next not suitable frame, so the previous is taken
            timestamps.append([face_id, time_start, time_end, frame_start, frame_end-1])
        
    
    if flag:
        result_faces[newkey] = new_dset
        newkey += 1
        face_id += 1
    
print('Total time:', total_time)

0
125.459 129.796 0 104 3009.0 3112.0
1
206.874 210.878 24 120 4961.0 5056.0
2
250.083 253.212 4 79 5997.0 6071.0
3
265.265 276.568 0 271 6361.0 6631.0
4
5
328.245 331.456 0 77 7871.0 7947.0
6
334.334 337.963 0 87 8017.0 8103.0
7
8
9
362.696 371.83 0 219 8697.0 8915.0
10
440.107 445.696 0 134 10553.0 10686.0
11
12
456.123 459.209 0 74 10937.0 11010.0
13
14
506.965 510.177 19 96 12156.0 12232.0
15
510.177 516.183 0 144 12233.0 12376.0
16
510.177 516.183 0 144 12233.0 12376.0
17
18
523.524 527.069 0 85 12553.0 12637.0
19
530.239 533.825 17 103 12714.0 12799.0
20
533.867 544.711 0 260 12801.0 13060.0
21
551.301 554.346 2 75 13219.0 13291.0
22
23
602.269 609.067 0 163 14441.0 14603.0
24
612.487 618.744 11 161 14686.0 14835.0
619.786 627.461 186 370 14861.0 15044.0
25
633.634 638.347 0 113 15193.0 15305.0
26
638.639 650.234 0 278 15313.0 15590.0
27
28
665.666 668.836 0 76 15961.0 16036.0
29
669.628 673.048 15 97 16056.0 16137.0
30
31
687.02 691.066 0 97 16473.0 16569.0
32
692.109 697.573 2 

In [18]:
timestamps

[[0, 125.459, 129.796, 3009.0, 3112.0],
 [1, 206.874, 210.878, 4961.0, 5056.0],
 [2, 250.083, 253.212, 5997.0, 6071.0],
 [3, 265.265, 276.568, 6361.0, 6631.0],
 [4, 328.245, 331.456, 7871.0, 7947.0],
 [5, 334.334, 337.963, 8017.0, 8103.0],
 [6, 362.696, 371.83, 8697.0, 8915.0],
 [7, 440.107, 445.696, 10553.0, 10686.0],
 [8, 456.123, 459.209, 10937.0, 11010.0],
 [9, 506.965, 510.177, 12156.0, 12232.0],
 [10, 510.177, 516.183, 12233.0, 12376.0],
 [11, 510.177, 516.183, 12233.0, 12376.0],
 [12, 523.524, 527.069, 12553.0, 12637.0],
 [13, 530.239, 533.825, 12714.0, 12799.0],
 [14, 533.867, 544.711, 12801.0, 13060.0],
 [15, 551.301, 554.346, 13219.0, 13291.0],
 [16, 602.269, 609.067, 14441.0, 14603.0],
 [17, 612.487, 618.744, 14686.0, 14835.0],
 [17, 619.786, 627.461, 14861.0, 15044.0],
 [18, 633.634, 638.347, 15193.0, 15305.0],
 [19, 638.639, 650.234, 15313.0, 15590.0],
 [20, 665.666, 668.836, 15961.0, 16036.0],
 [21, 669.628, 673.048, 16056.0, 16137.0],
 [22, 687.02, 691.066, 16473.0, 1656

In [19]:
#create a dataset with cleaned data

output = pd.DataFrame(columns=cols)

for i in result_faces:
    result_faces[i]['face_id'].values[:] = i
    output = pd.concat([output, result_faces[i]])

In [22]:
#write a clean dataset to the file
pd.DataFrame.from_dict(output).to_csv(f'./12movies_selected_frames/suitable_frames_{NAME}.csv', sep=',', mode='w')

In [349]:
#write timestamps to the file
output = pd.DataFrame(np.array(timestamps), columns=['face_id', 'time_start', 'time_end', 'frame_start', 'frame_end'])

In [350]:
pd.DataFrame.from_dict(output).to_csv(f'./12movies_output/timestamps_{NAME}.csv', sep=',', mode='w')

In [ ]:
#find start/end at the suitable dataframes
'''timestamps = []

for i in result_faces:
    temp = []
    id = int(result_faces[i].iloc[0]['face_id'])
    start = float(result_faces[i].iloc[0][['timestamp']])
    start_frame = float(result_faces[i].iloc[0][['frame']])
    end = float(result_faces[i].iloc[-1]['timestamp'])
    end_frame = float(result_faces[i].iloc[-1][['frame']])
    temp.append(id)
    temp.append(start)
    temp.append(end)
    temp.append(start_frame)
    temp.append(end_frame)

    timestamps.append(temp)
    print(temp)
'''